In [1]:
import pandas as pd

In [2]:
from gensim.models.ldamodel import LdaModel
from gensim.models.callbacks import CoherenceMetric
from gensim import corpora
from gensim.models.callbacks import PerplexityMetric

# import logging
# logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [4]:
data = pd.read_excel("../result/KP_ESG_sort_0203.xlsx")

In [5]:
data

,Unnamed: 0,key,url,title,body,comments
0,16260,023/0003589070?,https://n.news.naver.com/mnews/article/023/000...,“코로나로 이익 봤으면 나눠야” 이낙연 이익공유제 꺼냈다,[\n\n\n\n\n더불어민주당 이낙연대표가 11일 서울 여의도 국회에서 열린 최고...,5574
1,6804,011/0003517888?,https://n.news.naver.com/mnews/article/011/000...,뒷북경제 신용카드 소득공제·경유세···‘서민증세’ 반발 넘을 수 있을까,[\n\t\t\t[서울경제] 정부가 신용카드 소득공제를 줄이고 미세먼지 대책으로 경...,2081
2,11459,015/0004772552?,https://n.news.naver.com/mnews/article/015/000...,그날 하루에만 300만원 날렸는데… 떡볶이집 사장 울분,"[\n카카오 먹통 피해 '외식·서비스업'에 집중외식업, 페이 결제 피해 크고 톡 채...",839
3,6113,009/0004879440?,https://n.news.naver.com/mnews/article/009/000...,캐시백 10만원 받을줄 알았는데…통장보고 깜짝 놀랐다,[\n810만명에 3875억 첫 지급1인당 평균 4만8천원 받아169만명은 한도 1...,759
4,17027,025/0003133356?,https://n.news.naver.com/mnews/article/025/000...,與는 좌시 않겠다…규제 강화에 네이버·카카오 주가 급락,[\n \n\n\n\n여당인 더불어민주당(민주당)에서 인터넷플랫폼 기업의 시장 독...,745
...,...,...,...,...,...,...
3708,16965,025/0003085470?,https://n.news.naver.com/mnews/article/025/000...,팩플 네이버의 ‘오프라인 상륙작전’…이마트 타고 페이ㆍ명품 영토 확장,[\n\t\t\t이마트를 타고 상륙한 ‘쓱 랜더스’는 네이버였다. 16일 네이버와 ...,0
3709,17002,025/0003114599?,https://n.news.naver.com/mnews/article/025/000...,'제15회 국가지속가능경영 컨퍼런스' 지속가능금융·제품책임 부문 우수기업은?,"[\n\t\t\t한국언론인협회가 주최하고 기획재정부, 외교부, 보건복지부, 환경부,...",0
3710,17002,025/0003114599?,https://n.news.naver.com/mnews/article/025/000...,'제15회 국가지속가능경영 컨퍼런스' 지속가능금융·제품책임 부문 우수기업은?,"[\n\t\t\t한국언론인협회가 주최하고 기획재정부, 외교부, 보건복지부, 환경부,...",0
3711,17002,025/0003114599?,https://n.news.naver.com/mnews/article/025/000...,'제15회 국가지속가능경영 컨퍼런스' 지속가능금융·제품책임 부문 우수기업은?,"[\n\t\t\t한국언론인협회가 주최하고 기획재정부, 외교부, 보건복지부, 환경부,...",0


In [6]:
from konlpy.tag import Okt

In [7]:
okt = Okt()

In [9]:
data['token'] = data['body'].apply(lambda x : okt.nouns(x))

### 본문으로 분류

In [10]:
tokens = data['token']

In [11]:
dictionary = corpora.Dictionary(tokens)

In [12]:
dictionary.filter_extremes(no_below=2, no_above=0.5)

In [13]:
corpus = [dictionary.doc2bow(text) for text in tokens]

In [48]:
num_topics = 15
chunksize = 2000
passes = 20
iterations = 1000
eval_every = None

temp = dictionary[0]
id2word = dictionary.id2token

model = LdaModel(
    corpus=corpus,
    id2word=id2word,
    chunksize=chunksize,
    alpha='auto',
    eta='auto',
    iterations=iterations,
    num_topics=num_topics,
    passes=passes,
    eval_every=eval_every
)

/home/mglee/anaconda3/envs/save/lib/python3.8/site-packages/gensim/models/ldamodel.py:141: RuntimeWarning: divide by zero encountered in true_divide
  b = np.sum(gradf / q) / (1 / c + np.sum(1 / q))


In [49]:
import pickle
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis
from gensim.models.coherencemodel import CoherenceModel
import matplotlib.pyplot as plt

In [50]:
lda_visualization = gensimvis.prepare(model, corpus, dictionary, sort_topics=False)
pyLDAvis.save_html(lda_visualization, '../result/LDA_naver_ESG.html')

/home/mglee/anaconda3/envs/save/lib/python3.8/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(
/home/mglee/anaconda3/envs/save/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/home/mglee/anaconda3/envs/save/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/home/mglee/anaconda3/envs/save/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/home/mglee/anaconda3

In [51]:
topics = model.print_topics(num_words=10)
for topic in topics:
    print(topic)

(0, '0.012*"정부" + 0.010*"코로나" + 0.008*"경제" + 0.007*"우리" + 0.006*"그" + 0.005*"생각" + 0.005*"문제" + 0.005*"정책" + 0.005*"한국" + 0.005*"지금"')
(1, '0.035*"대표" + 0.010*"사회" + 0.009*"센터" + 0.008*"경영" + 0.008*"논란" + 0.007*"의장" + 0.007*"글로벌" + 0.006*"상생" + 0.006*"미래" + 0.006*"내정자"')
(2, '0.012*"상품" + 0.008*"대표" + 0.008*"보험" + 0.007*"출시" + 0.007*"고객" + 0.006*"데이터" + 0.006*"판매" + 0.006*"제품" + 0.005*"브랜드" + 0.005*"진행"')
(3, '0.039*"대출" + 0.018*"은행" + 0.014*"금리" + 0.012*"신용" + 0.012*"상품" + 0.010*"뱅크" + 0.009*"중" + 0.007*"사업자" + 0.007*"파이" + 0.006*"토스"')
(4, '0.054*"매출" + 0.045*"대비" + 0.035*"증가" + 0.032*"년" + 0.022*"동기" + 0.018*"기록" + 0.017*"영업" + 0.016*"분기" + 0.015*"콘텐츠" + 0.015*"부문"')
(5, '0.014*"환경" + 0.013*"경영" + 0.011*"사회" + 0.010*"탄소" + 0.010*"추진" + 0.010*"친환경" + 0.010*"위원회" + 0.007*"관련" + 0.007*"한국" + 0.006*"개인"')
(6, '0.029*"피해" + 0.020*"보험" + 0.017*"데이터" + 0.016*"센터" + 0.016*"보상" + 0.013*"톡" + 0.010*"상공" + 0.009*"화재" + 0.009*"장애" + 0.009*"사태"')
(7, '0.024*"카드" + 0.016*"할인" + 0.014*"혜택" + 0.013

In [52]:
import numpy as np

topic_lst = []
for i, topic_list in enumerate(model[corpus]):
    temp = pd.DataFrame(topic_list)
    max_idx = np.argmax(temp.iloc[:,1])
    topic_lst.append(temp.iloc[max_idx,0])

In [53]:
data['topic'] = topic_lst

In [56]:
data.sort_values("topic").to_excel("../result/KP_naverTB_LDA.xlsx")

### 기사 본문으로 분류

In [19]:
from konlpy.tag import Okt

In [13]:
from knockknock import discord_sender

url = "https://discord.com/api/webhooks/989774347466047529/c5ZeC8lk3gX8Jq8i77WVuIByZhPyNbTwLPIaY-zQEiH9flKeNcWb6hyUODRUFO0sfnKm"

In [20]:
@discord_sender(webhook_url = url)
def tokenization(lst, x):
    okt = Okt()
    for item in x:
        lst.append(okt.nouns(str(item)))
    return {"done"}

In [16]:
arti = pd.read_csv("../data/KP_daum_monthlyTop800_body.csv")

In [25]:
arti_token_lst = []
tokenization(arti_token_lst, arti['2'])

{'done'}

In [27]:
#arti_token_lst

In [28]:
arti_dictionary = corpora.Dictionary(arti_token_lst)

In [29]:
arti_dictionary.filter_extremes(no_below=2, no_above=0.5)

In [30]:
arti_corpus = [arti_dictionary.doc2bow(text) for text in arti_token_lst]

In [31]:
num_topics = 10
chunksize = 2000
passes = 20
iterations = 400
eval_every = None

temp = arti_dictionary[0]
id2word = arti_dictionary.id2token

model_arti = LdaModel(
    corpus=corpus,
    id2word=id2word,
    chunksize=chunksize,
    alpha='auto',
    eta='auto',
    iterations=iterations,
    num_topics=num_topics,
    passes=passes,
    eval_every=eval_every
)

/home/mglee/anaconda3/envs/save/lib/python3.8/site-packages/gensim/models/ldamodel.py:141: RuntimeWarning: divide by zero encountered in true_divide
  b = np.sum(gradf / q) / (1 / c + np.sum(1 / q))


In [33]:
lda_arti_visualization = gensimvis.prepare(model_arti, corpus, arti_dictionary, sort_topics=False)
pyLDAvis.save_html(lda_arti_visualization, '../result/LDA_arti.html')

/home/mglee/anaconda3/envs/save/lib/python3.8/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(
/home/mglee/anaconda3/envs/save/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/home/mglee/anaconda3/envs/save/lib/python3.8/site-packages/joblib/backports.py:36: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(np.__version__) < '1.13':
/home/mglee/anaconda3/envs/save/lib/python3.8/site-packages/setuptools/_distutils/version.py:346: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  other = LooseVersion(other)
/home/mglee/anaconda3/envs/save/lib/python3.8/site-